Querying file and Extracting data 

In [0]:
SELECT * ,_metadata.file_path as path FROM read_files('/Volumes/gizmobox/landing/operational_data/customers/*.json')

In [0]:
create or replace view gizmobox.bronze.v_customers
as SELECT * ,_metadata.file_path as path FROM read_files('/Volumes/gizmobox/landing/operational_data/customers/')

In [0]:
select current_catalog();
USE catalog gizmobox;
use schema bronze;
select * from v_customers

Complex and faulty JSON files

In [0]:
SELECT * FROM read_files( '/Volumes/gizmobox/landing/operational_data/orders/', format => 'text' );

In [0]:
SELECT * FROM read_files( '/Volumes/gizmobox/landing/operational_data/orders/', format => 'json', mode => 'PERMISSIVE', columnNameOfCorruptRecord => '_corrupt_record' );

In [0]:
%python
df_bronze = (
    spark.read
        .format("text")
        .load("/Volumes/gizmobox/landing/operational_data/orders/")
)

df_bronze.write.format("delta").mode("append").save("/mnt/bronze/raw_json")


Querying Binary files such as PNG files

In [0]:
%fs ls '/Volumes/gizmobox/landing/operational_data/memberships/2025-01'

In [0]:
SELECT * FROM read_files( '/Volumes/gizmobox/landing/operational_data/memberships' );

In [0]:
select * from binaryFile.`Volumes/gizmobox/landing/operational_data/memberships`

In [0]:
SELECT *
FROM read_files(
  'dbfs:/Volumes/gizmobox/landing/operational_data/memberships',
  format => 'binaryFile'
);


In [0]:
create or replace view gizmobox.bronze.v_memberships
as SELECT * FROM read_files(
  '/Volumes/gizmobox/landing/operational_data/memberships',
  format => 'binaryFile'
);
select * from gizmobox.bronze.v_memberships

reading CSV files

In [0]:
select * from read_files('/Volumes/gizmobox/landing/operational_data/addresses', format => 'csv', header => true, delimiter => '\t');

In [0]:
create or replace view gizmobox.bronze.v_addresses
as select * from read_files('/Volumes/gizmobox/landing/operational_data/addresses', format => 'csv', header => true, delimiter => '\t');
select * from gizmobox.bronze.v_addresses

In [0]:
%fs ls  'abfss://gizmobox@daecourseexdl.dfs.core.windows.net/landing/external_data/payments';


In [0]:
CREATE TABLE gizmobox.bronze.payments
(payment_id integer,
order_id integer,
payment_timestamp timestamp,
payment_status integer,
payment_method string
)
using csv
options (header=false, delimiter=',')
location 'abfss://gizmobox@daecourseexdl.dfs.core.windows.net/landing/external_data/payments'


In [0]:
drop table gizmobox.bronze.payments

In [0]:
select * from gizmobox.bronze.payments